In [1]:
import numpy as np
import subprocess 
import cv2
import time
import matplotlib.pyplot as plt

def LoadImages(Dataset):  # Load images from training or testing dataset 
    ClassEncs = ["building", "car", "mountain", "tree", "beach"]
    if (Dataset=="Train"):
        Paths, Imgs = ([], [])
        for ClassEnc in ClassEncs:
            # List all images
            ImgDir = "imagesDatabaseHW7/training/%s/" %ClassEnc
            out, _ = subprocess.Popen("ls "+ImgDir, shell=True,
                                      stdout=subprocess.PIPE,
                                      stderr=subprocess.PIPE).communicate()
            Paths = Paths + [ImgDir + x.decode("utf-8") for x in out.split()]
        GT = [0]*20 + [1]*20 + [2]*20 + [3]*20 + [4]*20        
    else: # Dataset=="Test"
        Paths, Imgs = ([], [])
        for ClassEnc in ClassEncs:
            # List all images           
            cmd = "ls imagesDatabaseHW7/testing/%s_*" %ClassEnc
            out, _  = subprocess.Popen(cmd, shell=True,
                                       stdout=subprocess.PIPE,
                                       stderr=subprocess.PIPE).communicate()
            Paths = Paths + [x.decode("utf-8") for x in out.split()]
        GT = [0]*5 + [1]*5 + [2]*5 + [3]*5 + [4]*5
    for Path in Paths:
        # Read images
        Img = cv2.imread(Path, cv2.IMREAD_GRAYSCALE) # (H, W) uint8 np ndarray
        Imgs.append(Img)
    return Imgs, Paths, GT # list of (2D uint8, str, int)
def LBP_Parameters(R = 1, P = 8): # Init parameters in LBP
    dh = np.zeros((P*4, )).astype("int") # 1st dim
    dw = np.zeros((P*4, )).astype("int") # 2nd dim
    # Each 4 colomns is the multipler in bilinear interpolatation
    BLI_Matrix = np.zeros((P*4, P)) 
    for idx_P in range(P):
        h = R * np.cos(2*np.pi*idx_P/P)
        w = R * np.sin(2*np.pi*idx_P/P) 
        dh[4*idx_P] = np.floor(h).astype("int")        # A
        dh[4*idx_P+1] = np.floor(h).astype("int")      # B
        dh[4*idx_P+2] = (np.floor(h)+1).astype("int")  # C
        dh[4*idx_P+3] = (np.floor(h)+1).astype("int")  # D
        dw[4*idx_P] = np.floor(w).astype("int")        # A
        dw[4*idx_P+1] = (np.floor(w)+1).astype("int")  # B
        dw[4*idx_P+2] = np.floor(w).astype("int")      # C
        dw[4*idx_P+3] = (np.floor(w)+1).astype("int")  # D
        dv, du = (h-np.floor(h), w-np.floor(w))
        BLI_Matrix[4*idx_P, idx_P] = (1-du)*(1-dv)     # A
        BLI_Matrix[4*idx_P+1, idx_P] = (1-du)*dv       # B 
        BLI_Matrix[4*idx_P+2, idx_P] = du*(1-dv)       # C
        BLI_Matrix[4*idx_P+3, idx_P] = du*dv           # D
    return BLI_Matrix, dh, dw
def LBP(Imgs, R = 1, P = 8): # Img is 2D grey-scale image
    # Init with parameters
    BLI_Matrix, dh, dw = LBP_Parameters(R, P)
    ImgHists = np.zeros((P+2, len(Imgs)))
    
    for idx, Img in enumerate(Imgs):
        #print(idx, end=",")
        
        # Step 1a: Construct a (H*W, P*4) matrix of the 4-neighbor of each 8 points
        ImgInterp0 = np.zeros((int((Img.shape[0]-2*R-2)*(Img.shape[1]-2*R-2)), P*4))
        for idx_p0, (dh0, dw0) in enumerate(zip(dh, dw)):
            ImgInterp0[:,idx_p0] = Img[R+1+dh0:Img.shape[0]-R-1+dh0, 
                                       R+1+dw0:Img.shape[1]-R-1+dw0].flatten()
            
        # Step 1b: ImgInterp0 * BLI_Matrix, then unflatten and get the raw LBP
        #          Output shape: (H-2R-2, H-2R-2, P)
        ImgInterp = np.matmul(ImgInterp0, BLI_Matrix)
        ImgInterp = np.reshape(ImgInterp, (int((Img.shape[0]-2*R-2)), 
                                           int((Img.shape[1]-2*R-2)), P))
        # Step 2: Compare ImgInterp with Img, output shape: (H-2R-2, H-2R-2, P)
        RawImg = np.expand_dims(Img[R+1:Img.shape[0]-R-1, 
                                    R+1:Img.shape[1]-R-1], axis=-1)
        ImgBin = (ImgInterp - RawImg) > 0  
        
        # Step 3: Get MinIntVal
        ImgIntVal = np.zeros((int(Img.shape[0]-2*R-2), int(Img.shape[1]-2*R-2)))
        ImgRotIntVal = np.zeros((int(Img.shape[0]-2*R-2), int(Img.shape[1]-2*R-2), P))
        ImgMinIntVal = np.zeros((int(Img.shape[0]-2*R-2), int(Img.shape[1]-2*R-2)))
        for idx_P in range(P):
            ImgIntVal = ImgIntVal + ImgBin[:, :, idx_P] * (2**idx_P)
        ImgIntVal = ImgIntVal.astype("int")
        for idx_P in range(P):
            ImgRotIntVal[:, :, idx_P] = rol(ImgIntVal, idx_P, P)
        ImgMinIntVal = np.min(ImgRotIntVal, axis = -1)

        # Step 4: Encode the MinIntVal using "runs"
        ImgEnc = np.zeros(ImgMinIntVal.shape)
        ImgEncOcc = np.zeros(ImgMinIntVal.shape)
        for idx_P in range(P+1): # There is idx_P 1's followed by 0
            MaskNumber = (2**idx_P)-1
            ImgEnc = ImgEnc + (ImgMinIntVal==MaskNumber)*idx_P
            ImgEncOcc = ImgEncOcc + (ImgMinIntVal==MaskNumber)
        ImgEnc = ImgEnc + (1-ImgEncOcc)*(P+1)
        
        # Step 5: Get histogram
        histogram_bin =  np.arange(-0.5, P+1+0.5+1e-3, 1) # (P+3)-long, P+2 bins
        hist, _ = np.histogram(ImgEnc, bins = histogram_bin)
        ImgHists[:, idx] = hist/sum(hist)
    return ImgHists
# Bit-wise rotate left: 0b00001001 --> 0b00010010
rol = lambda val, l_bits, max_bits: \
    (val << l_bits) & (2**max_bits-1) | \
    ((val & (2**max_bits-1)) >> (max_bits-l_bits))

In [2]:
(R, P) = (1, 8)
# Train & Test
TrainImgs, TrainPaths, TrainingGT = LoadImages("Train")
Patterns_Train = LBP(TrainImgs, R, P)
TestImgs, TestPaths, TestGT = LoadImages("Test")
Patterns_Test = LBP(TestImgs, R, P)

for x in range(5):
    print("%s: %.3f %.3f %.3f %.3f %.3f %.3f %.3f %.3f %.3f %.3f" 
          %(TrainPaths[x*20], Patterns_Train[0, x*20], Patterns_Train[1, x*20], Patterns_Train[2, x*20], 
           Patterns_Train[3, x*20], Patterns_Train[4, x*20], Patterns_Train[5, x*20],
           Patterns_Train[6, x*20], Patterns_Train[7, x*20], Patterns_Train[8, x*20],
           Patterns_Train[9, x*20]))

# Matching patterns
ConfMatrix = np.zeros((5, 5))
N_Train, N_Test = (len(TrainImgs), len(TestImgs))
for idx_testimg in range(N_Test):
    # Compute Distance
    EU_Dis = np.zeros((N_Train,))
    for idx_trainimg in range(N_Train):
        EU_Dis[idx_trainimg] = np.sqrt(np.sum((Patterns_Test[:,idx_testimg] - 
                                               Patterns_Train[:,idx_trainimg])**2))

    # Sort in increasing order (EU_Dis, TrainingGT)
    Sorted_EU_Dis, Sorted_TrainingGT = zip(*sorted(zip(EU_Dis, TrainingGT)))
    Sorted_EU_Dis = np.array(Sorted_EU_Dis)
    Sorted_TrainingGT = np.array(Sorted_TrainingGT)

    # The GroundTruth is TestGT[idx_testimg]
    ClassScore = np.zeros((5,))
    for idx_cls in range(5):
        Mask = (Sorted_TrainingGT[0:5]==idx_cls)
        ClassScore[idx_cls] = np.sum(Mask)*100 + 1 - np.sum(Sorted_EU_Dis[0:5]*Mask)
    Prediction = np.argmax(ClassScore) 
    ConfMatrix[TestGT[idx_testimg], Prediction] = ConfMatrix[TestGT[idx_testimg], Prediction] + 1
ConfMatrixDiag = [ConfMatrix[x,x] for x in range(5)]
print("ConfMatrix = \n", ConfMatrix.astype("int"))
print("(R, P) = (%d, %d), Test accuracy = %.0f%%" %(R, P, np.sum(ConfMatrixDiag)/25*100))

imagesDatabaseHW7/training/building/01.jpg: 0.134 0.099 0.038 0.100 0.099 0.056 0.024 0.083 0.076 0.292
imagesDatabaseHW7/training/car/01.jpg: 0.114 0.307 0.034 0.099 0.093 0.058 0.027 0.056 0.063 0.148
imagesDatabaseHW7/training/mountain/01.jpg: 0.208 0.191 0.041 0.071 0.057 0.050 0.036 0.056 0.083 0.206
imagesDatabaseHW7/training/tree/01.jpg: 0.145 0.081 0.054 0.079 0.071 0.073 0.050 0.082 0.145 0.222
imagesDatabaseHW7/training/beach/10.jpg: 0.190 0.111 0.043 0.124 0.094 0.080 0.037 0.060 0.084 0.177
ConfMatrix = 
 [[3 0 0 2 0]
 [0 3 0 0 2]
 [0 0 3 0 2]
 [1 0 0 4 0]
 [0 0 1 0 4]]
(R, P) = (1, 8), Test accuracy = 68%


In [3]:
# Internal view of wrong predictions
subprocess.call(["mkdir", "WrongPrediction"])
for idx_testimg in range(N_Test):
    # Compute Distance
    EU_Dis = np.zeros((N_Train,))
    for idx_trainimg in range(N_Train):
        EU_Dis[idx_trainimg] = np.sqrt(np.sum((Patterns_Test[:,idx_testimg] - 
                                               Patterns_Train[:,idx_trainimg])**2))

    # Sort in increasing order (EU_Dis, TrainingGT)
    Sorted_EU_Dis, Sorted_TrainingGT = zip(*sorted(zip(EU_Dis, TrainingGT)))
    Sorted_EU_Dis = np.array(Sorted_EU_Dis)
    Sorted_TrainingGT = np.array(Sorted_TrainingGT)

    # The GroundTruth is TestGT[idx_testimg]
    ClassScore = np.zeros((5,))
    for idx_cls in range(5):
        Mask = (Sorted_TrainingGT[0:5]==idx_cls)
        ClassScore[idx_cls] = np.sum(Mask)*100 + 1 - np.sum(Sorted_EU_Dis[0:5]*Mask)
    Prediction = np.argmax(ClassScore) 
    ClassEncs = ["building", "car", "mountain", "tree", "beach"]
    if(TestGT[idx_testimg] != Prediction):
        print("%s: GT = %d, Prediction = %d" %(TestPaths[idx_testimg], TestGT[idx_testimg], Prediction))
        newpath = "WrongPrediction/%s_pred_%s.jpg" \
              %(TestPaths[idx_testimg].split("/")[-1].split(".")[0], ClassEncs[Prediction])
        subprocess.call(["cp", TestPaths[idx_testimg], newpath])

imagesDatabaseHW7/testing/building_4.jpg: GT = 0, Prediction = 3
imagesDatabaseHW7/testing/building_5.jpg: GT = 0, Prediction = 3
imagesDatabaseHW7/testing/car_2.jpg: GT = 1, Prediction = 4
imagesDatabaseHW7/testing/car_3.jpg: GT = 1, Prediction = 4
imagesDatabaseHW7/testing/mountain_1.jpg: GT = 2, Prediction = 4
imagesDatabaseHW7/testing/mountain_5.jpg: GT = 2, Prediction = 4
imagesDatabaseHW7/testing/tree_5.jpg: GT = 3, Prediction = 0
imagesDatabaseHW7/testing/beach_2.jpg: GT = 4, Prediction = 2
